# <div align="center" style="color: #ff5733;">New KPI Report</div>

## *Requestor is Jennyfer & Anne*

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

# FPD30 April 2024 Month End

In [3]:
sq = """SELECT t1.new_loan_type,
ROUND(SUM(CASE when t1.defFPD30 = 1 then 1 ELSE 0 end )/sum(t1.obsFPD30)*100,3) as percentage_of_FPD30
from `prj-prod-dataplatform.risk_credit_mis.loan_master_table` t1
inner JOIN prj-prod-dataplatform.risk_credit_mis.loan_bucket_flow_report_core t2
ON
t1.loanAccountNumber = t2.loanAccountNumber
WHERE t1.flagDisbursement = 1
and t1.new_loan_type in ('Quick','Flex','SIL-Instore')
and t2.loanStatus in ('In Arrears','Normal')
and t2.bucketDate in ('2024-04-30')
GROUP BY 1;"""

april24fpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
april24fpd30

Job ID b12d0e13-6524-4183-aff5-5c4eeb0ca7e9 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,new_loan_type,percentage_of_FPD30
0,SIL-Instore,5.932
1,Quick,9.310
2,Flex,1.199


# 30+, 60+, 90+ for April 2024

In [4]:
sq = """with b as
(select t1.new_loan_type 
, count(distinct t2.loanAccountNumber) cnt_loan
, count(distinct case when t2.Max_current_DPD >=30 then t2.loanAccountNumber end) thirtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=30 then t2.loanAccountNumber end) obsthirtyplus
, count(distinct case when t2.Max_current_DPD >=60 then t2.loanAccountNumber end) sixtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=60 then t2.loanAccountNumber end) obssixtyplus
, count(distinct case when t2.Max_current_DPD >=90 then t2.loanAccountNumber end) nintyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >= 90 then t2.loanAccountNumber end) obsnintyplus
from `risk_credit_mis.loan_bucket_flow_report_core` t2
inner join `risk_credit_mis.loan_master_table` t1 on t2.loanAccountNumber = t1.loanAccountNumber
WHERE t1.flagDisbursement = 1
and t1.new_loan_type in ('Quick','Flex','SIL-Instore')
and t2.loanStatus in ('In Arrears','Normal')
and t2.bucketDate in ('2024-04-30')
group by 1
)
select 
new_loan_type,
round(sum(thirtyplus)/sum(obsthirtyplus)*100, 3) thirtyplus,
round(sum(sixtyplus)/sum(obsthirtyplus)*100, 3) sixtyplus,
round(sum(nintyplus)/sum(obsnintyplus)*100, 3) nintyplus,
from b
group by 1
order by 1
;"""

secondpartApril24 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
secondpartApril24

Job ID ddfcdc79-56b7-4065-b13b-02e900456c04 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,new_loan_type,thirtyplus,sixtyplus,nintyplus
0,Flex,25.929,21.934,18.386
1,Quick,49.133,42.845,41.079
2,SIL-Instore,17.740,12.563,13.475


# 30+, 60+, 90+ for April 2024 (without Product)

In [6]:
sq = """with b as
(select 
count(distinct t2.loanAccountNumber) cnt_loan
, count(distinct case when t2.Max_current_DPD >=30 then t2.loanAccountNumber end) thirtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=30 then t2.loanAccountNumber end) obsthirtyplus
, count(distinct case when t2.Max_current_DPD >=60 then t2.loanAccountNumber end) sixtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=60 then t2.loanAccountNumber end) obssixtyplus
, count(distinct case when t2.Max_current_DPD >=90 then t2.loanAccountNumber end) nintyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >= 90 then t2.loanAccountNumber end) obsnintyplus
from `risk_credit_mis.loan_bucket_flow_report_core` t2
inner join `risk_credit_mis.loan_master_table` t1 on t2.loanAccountNumber = t1.loanAccountNumber
WHERE t1.flagDisbursement = 1
and t1.new_loan_type in ('Quick','Flex','SIL-Instore')
and t2.loanStatus in ('In Arrears','Normal')
and t2.bucketDate in ('2024-04-30')
)
select 
round(sum(thirtyplus)/sum(obsthirtyplus)*100, 3) thirtyplus,
round(sum(sixtyplus)/sum(obsthirtyplus)*100, 3) sixtyplus,
round(sum(nintyplus)/sum(obsnintyplus)*100, 3) nintyplus,
from b
order by 1
;"""

thirdpartapril24 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
thirdpartapril24

Job ID 79f34129-3c76-483c-9431-2359e6552745 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,thirtyplus,sixtyplus,nintyplus
0,24.545,19.229,20.881


In [7]:
dfapril24 = april24fpd30.merge(secondpartApril24, on='new_loan_type', how='left')
dfapril24

,new_loan_type,percentage_of_FPD30,thirtyplus,sixtyplus,nintyplus
0,SIL-Instore,5.932,17.740,12.563,13.475
1,Quick,9.310,49.133,42.845,41.079
2,Flex,1.199,25.929,21.934,18.386


In [8]:
resultapril24 = pd.concat([dfapril24, thirdpartapril24], ignore_index=True)
resultapril24

,new_loan_type,percentage_of_FPD30,thirtyplus,sixtyplus,nintyplus
0,SIL-Instore,5.932,17.740,12.563,13.475
1,Quick,9.310,49.133,42.845,41.079
2,Flex,1.199,25.929,21.934,18.386
3,NaN,NaN,24.545,19.229,20.881


In [9]:
resultapril24['new_loan_type'] = resultapril24['new_loan_type'].fillna("Portfolio")
resultapril24

,new_loan_type,percentage_of_FPD30,thirtyplus,sixtyplus,nintyplus
0,SIL-Instore,5.932,17.740,12.563,13.475
1,Quick,9.310,49.133,42.845,41.079
2,Flex,1.199,25.929,21.934,18.386
3,Portfolio,NaN,24.545,19.229,20.881


In [10]:
import pandas as pd
from openpyxl import load_workbook


In [14]:
# Load the existing Excel file
existing_excel_file = r'C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Collection\Jennifer Zaldivar\KRI_New_Report\Data\20240411_KRI_Report_New.xlsx'
xls = pd.ExcelFile(existing_excel_file)

# Read each existing sheet into separate dataframes
df_january = pd.read_excel(xls, sheet_name='January2024')
df_february = pd.read_excel(xls, sheet_name='February2024')
df_march = pd.read_excel(xls, sheet_name='March2024')


# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter(r'C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Collection\Jennifer Zaldivar\KRI_New_Report\Data\20240502_KRI_Report_New.xlsx', engine='xlsxwriter')

# Write each dataframe to a separate worksheet
df_january.to_excel(writer, sheet_name='January2024')
df_february.to_excel(writer, sheet_name='February2024')
df_march.to_excel(writer, sheet_name='March2024')
resultapril24.to_excel(writer, sheet_name='April2024')

# Close the Pandas Excel writer and output the Excel file
writer.close()
